In [1]:
!pip install pyspellchecker


   ---------------------------------------- 0.0/7.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.1 MB 1.2 MB/s eta 0:00:06
   ---- ----------------------------------- 0.8/7.1 MB 1.2 MB/s eta 0:00:06
   ----- ---------------------------------- 1.0/7.1 MB 1.2 MB/s eta 0:00:06
   ------- -------------------------------- 1.3/7.1 MB 1.2 MB/s eta 0:00:06
   -------- ------------------------------- 1.6/7.1 MB 1.1 MB/s eta 0:00:05
   ---------- ----------------------------- 1.8/7.1 MB 1.2 MB/s eta 0:00:05
   ----------- ---------------------------- 2.1/7.1 MB 1.2 MB/s eta 0:00:05
   ------------- -------------------------- 2.4/7.1 MB 1.2 MB/s eta 0:00:05
   -------------- ------------------------- 2.6/7.1 MB 1.2 MB/s eta 0:00:04
   ---------------- ----------------------- 2.9/7.1 MB 1.2 MB/s eta 0:00:04
   ----------------- ------------

In [9]:
!pip install pytesseract
!apt-get install -y tesseract-ocr
!sudo apt-get install tesseract-ocr-nep

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,660 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [1]:
import torch
from collections import Counter
from transformers import AutoModelForTokenClassification, AutoTokenizer


model = AutoModelForTokenClassification.from_pretrained("../model/V1/saved_model")
tokenizer = AutoTokenizer.from_pretrained("../model/V1/tokenizer_saved_model")



c:\Users\Dell\miniconda3\envs\minor\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import re
import string
from spellchecker import SpellChecker
import pytesseract
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

# Initialize the spellchecker.
#spell = SpellChecker()


In [3]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
os.environ['TESSDATA_PREFIX'] = r'C:\Program Files\Tesseract-OCR\tessdata'
custom_config = r'--oem 3 --psm 6'

In [4]:
tags = ["O", "CIT-NUM", "SEX", "B-NAME", "I-NAME", "YEAR", "MONTH", "DAY", "B-DISTRICT", "B-WARD", "B-NO", "I-DISTRICT", "I-WARD", "I-NO"]
id2label = {i: label for i, label in enumerate(tags)}
label2id = {v: k for k, v in id2label.items()}

In [17]:
image_back = cv2.imread("../data/back1.jpg")
back_text = pytesseract.image_to_string(image_back, config=custom_config)
unwanted = ";:{}[]()|,~=+_-—.«*‘“"
translator = str.maketrans('', '', unwanted)
cleaned_text = back_text.translate(translator)
try:
    try :
        start_index = cleaned_text.lower().index("details")
        truncated_text = cleaned_text[start_index:]

    except (ValueError, StopIteration):
        start_index = cleaned_text.lower().index("government")
        truncated_text = cleaned_text[start_index:]


    split_text = truncated_text.split()


    ward_index = next(i for i, word in enumerate(split_text) if word.lower() == 'ward')


    result_words = split_text[:ward_index +25]
    result = ' '.join(result_words)
except (ValueError, StopIteration):
    result = cleaned_text

print(result)


details Citizenship Certificate No 37027700954 Sex Male Full Name SAGAR GIRI Date of Birth AD Year2003 MonthJUN Day06 Birth lacs District Gulmi i selh werniraeras carn ox ue arf VDC Bamgha Ward No8 Permanent Addres> District Rupandehi pape teres? wr ae op ale il Municipality Tilottama Ward No1 dl AMS AAA Cet OES AST Ay erraTLarcTAT


In [11]:
import re
import string

def split_token(token):

    token = re.sub(r'([a-z])([A-Z])', r'\1 \2', token)
    parts = []
    for part in token.split():
        parts.extend(re.findall(r'[A-Za-z]+|\d+', part))
    return parts

def clean_text(text):

    raw_tokens = text.split()
    cleaned_tokens = []
    for token in raw_tokens:
        token = token.strip(string.punctuation)
        if not token:
            continue
        sub_tokens = split_token(token)
        cleaned_tokens.extend([sub for sub in sub_tokens if sub])
    return cleaned_tokens


ocr_text = result

cleaned_tokens = clean_text(ocr_text)
sentence = " ".join(cleaned_tokens)
print(sentence)


details Citizenship Certificate No 37027700054 Sex Male Full Names SAGARGIRI i Date of Birth AD Year 2003 Month JUN Day 06 Birth Placs CDs trict Guimi va srniranra dees tet rh VDC Bamnghs Ward No 8 i Perman nt Address S District Rupandehi sly pisare ce igs yl Municipality Tilottama Ward No 1 Boe aia Ca 2083 aig at aera WATT


In [12]:


text = sentence


inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

predicted_label_ids = outputs.logits.argmax(-1).squeeze()
predicted_labels = [id2label[id.item()] for id in predicted_label_ids]

tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids.squeeze())
word_ids = inputs.word_ids()


words = []
word_labels = []
current_word = ""
current_labels = []

for token, word_id, label in zip(tokens, word_ids, predicted_labels):
    if word_id is None:
        continue

    if word_id != len(words):

        if current_word:
            label_counts = Counter(current_labels)
            most_common_label = max(label_counts, key=label_counts.get)

            if "O" in label_counts and len(label_counts) > 1:
                del label_counts["O"]
                most_common_label = max(label_counts, key=label_counts.get)

            words.append((current_word, most_common_label))

        current_word = token.replace("##", "")
        current_labels = [label]
    else:

        current_word += token.replace("##", "")
        current_labels.append(label)


if current_word:
    label_counts = Counter(current_labels)
    most_common_label = max(label_counts, key=label_counts.get)

    if "O" in label_counts and len(label_counts) > 1:
        del label_counts["O"]
        most_common_label = max(label_counts, key=label_counts.get)

    words.append((current_word, most_common_label))

print(words)


[('details', 'O'), ('citizenship', 'O'), ('certificate', 'O'), ('no', 'O'), ('37027700054', 'CIT-NUM'), ('sex', 'O'), ('male', 'SEX'), ('full', 'O'), ('names', 'O'), ('sagargiri', 'B-NAME'), ('i', 'O'), ('date', 'O'), ('of', 'O'), ('birth', 'O'), ('ad', 'O'), ('year', 'O'), ('2003', 'YEAR'), ('month', 'O'), ('jun', 'MONTH'), ('day', 'O'), ('06', 'DAY'), ('birth', 'O'), ('placs', 'O'), ('cds', 'O'), ('trict', 'O'), ('guimi', 'B-NAME'), ('va', 'O'), ('srniranra', 'I-NAME'), ('dees', 'O'), ('tet', 'O'), ('rh', 'O'), ('vdc', 'O'), ('bamnghs', 'B-WARD'), ('ward', 'O'), ('no', 'B-NO'), ('8', 'B-NO'), ('i', 'O'), ('perman', 'O'), ('nt', 'O'), ('address', 'O'), ('s', 'O'), ('district', 'O'), ('rupandehi', 'B-DISTRICT'), ('sly', 'O'), ('pisare', 'I-DISTRICT'), ('ce', 'O'), ('igs', 'O'), ('yl', 'O'), ('municipality', 'O'), ('tilottama', 'I-WARD'), ('ward', 'O'), ('no', 'B-NO'), ('1', 'I-NO'), ('boe', 'O'), ('aia', 'O'), ('ca', 'O'), ('2083', 'I-NO'), ('aig', 'O'), ('at', 'O'), ('aera', 'O'), ('w

In [13]:
#this correxts the error if th I-district is before B-district then change it. so for ward
D_target = ["B-DISTRICT","I-DISTRICT"]
W_target = ["B-WARD","I-WARD"]
NO_target = ["B-NO","I-NO"]
D_flag = 0
W_flag = 0
NO_flag = 0

new_words = []

for data,label in words:
    
    if label in D_target and D_flag == 0:
        label = "B-DISTRICT"
        D_flag = 1

    elif label in D_target and D_flag == 1:
        label = "I-DISTRICT"

    elif label in W_target and W_flag == 0:
        label = "B-WARD"
        W_flag = 1

    elif label in W_target and W_flag == 1:
        label = "I-WARD"

    elif label in NO_target and NO_flag < 2:
        label = "B-NO"
        NO_flag += 1
    
    elif label in NO_target and NO_flag ==2:
        label = "I-NO"
    
    new_words.append((data,label))
    
print(new_words)


[('details', 'O'), ('citizenship', 'O'), ('certificate', 'O'), ('no', 'O'), ('37027700054', 'CIT-NUM'), ('sex', 'O'), ('male', 'SEX'), ('full', 'O'), ('names', 'O'), ('sagargiri', 'B-NAME'), ('i', 'O'), ('date', 'O'), ('of', 'O'), ('birth', 'O'), ('ad', 'O'), ('year', 'O'), ('2003', 'YEAR'), ('month', 'O'), ('jun', 'MONTH'), ('day', 'O'), ('06', 'DAY'), ('birth', 'O'), ('placs', 'O'), ('cds', 'O'), ('trict', 'O'), ('guimi', 'B-NAME'), ('va', 'O'), ('srniranra', 'I-NAME'), ('dees', 'O'), ('tet', 'O'), ('rh', 'O'), ('vdc', 'O'), ('bamnghs', 'B-WARD'), ('ward', 'O'), ('no', 'B-NO'), ('8', 'B-NO'), ('i', 'O'), ('perman', 'O'), ('nt', 'O'), ('address', 'O'), ('s', 'O'), ('district', 'O'), ('rupandehi', 'B-DISTRICT'), ('sly', 'O'), ('pisare', 'I-DISTRICT'), ('ce', 'O'), ('igs', 'O'), ('yl', 'O'), ('municipality', 'O'), ('tilottama', 'I-WARD'), ('ward', 'O'), ('no', 'I-NO'), ('1', 'I-NO'), ('boe', 'O'), ('aia', 'O'), ('ca', 'O'), ('2083', 'I-NO'), ('aig', 'O'), ('at', 'O'), ('aera', 'O'), ('w

In [ ]:
# #this correxts the error if th I-district is before B-district then change it. so for ward
# # Define our target labels for processing.
# target_labels = {
#     "DISTRICT": {"B-DISTRICT", "I-DISTRICT"},
#     "WARD": {"B-WARD", "I-WARD"},
#     "NO": {"B-NO", "I-NO"}
    
# }

# # Helper: Return the group if the label exactly matches one of our target labels.
# def get_group(label):
#     for group, labels in target_labels.items():
#         if label in labels:
#             return group
#     return None

# # Flag to indicate if we've reached the permanent section.
# in_permanent = False
# modified_words = []

# for word, label in words:
#     # Check if this token marks the start of permanent address.
#     if word.lower() == "permanent":
#         in_permanent = True
#         modified_words.append((word, label))
#         continue

#     group = get_group(label)
#     if group is not None:
#         # If we're in the permanent section, force I-; otherwise, force B-
#         if in_permanent:
#             new_label = "I-" + group
#         else:
#             new_label = "B-" + group
#         modified_words.append((word, new_label))
#     else:
#         modified_words.append((word, label))

# # Replace your original words list with the modified one.
# words = modified_words

# print(words)


[('details', 'O'), ('citizenship', 'O'), ('certificat', 'O'), ('no', 'O'), ('06147500089', 'CIT-NUM'), ('sex', 'O'), ('mate', 'SEX'), ('full', 'O'), ('name', 'O'), ('sabin', 'B-NAME'), ('rana', 'O'), ('magar', 'I-NAME'), ('date', 'O'), ('of', 'O'), ('birth', 'O'), ('ad', 'O'), ('year', 'O'), ('2002', 'YEAR'), ('month', 'O'), ('nov', 'MONTH'), ('day', 'O'), ('19', 'DAY'), ('birth', 'O'), ('place', 'O'), ('district', 'O'), ('sunsari', 'B-DISTRICT'), ('municipality', 'O'), ('barahachhetra', 'B-WARD'), ('ward', 'O'), ('no', 'B-NO'), ('8', 'B-NO'), ('permahent', 'O'), ('address', 'O'), ('district', 'O'), ('sunsari', 'B-DISTRICT'), ('municipality', 'O'), ('barahachhetra', 'B-WARD'), ('ward', 'O'), ('no', 'B-NO'), ('8', 'B-NO'), ('aut', 'O'), ('atta', 'B-WARD'), ('vat', 'O'), ('ez', 'O'), ('sansa', 'B-WARD'), ('ai', 'O'), ('aaah', 'O'), ('wats', 'O'), ('fesvenl', 'B-DISTRICT'), ('s', 'O'), ('arita', 'B-WARD'), ('paar', 'B-WARD'), ('deter', 'O'), ('ee', 'O'), ('eae', 'O')]


In [ ]:
# new_words = []
# tagss=[]
# for idx , (word, label) in enumerate(words):
#     tagss.append(label)
#     if label not in ['B-NO', 'I-NO','O']:
#         if tagss.count(label) > 1:
#             continue
#     new_words.append((word, label))

# print(new_words)
#its improved form is below
    

[('details', 'O'), ('citizenship', 'O'), ('certificate', 'O'), ('no', 'O'), ('31017603422', 'CIT-NUM'), ('sex', 'O'), ('male', 'SEX'), ('full', 'O'), ('name', 'O'), ('subash', 'B-NAME'), ('khatri', 'I-NAME'), ('date', 'O'), ('of', 'O'), ('birth', 'O'), ('ad', 'O'), ('year', 'O'), ('2001', 'YEAR'), ('month', 'O'), ('apr', 'MONTH'), ('day', 'O'), ('14', 'DAY'), ('birth', 'O'), ('place', 'O'), ('district', 'O'), ('makawanpur', 'B-DISTRICT'), ('municipality', 'O'), ('thaha', 'B-WARD'), ('ward', 'O'), ('no', 'B-NO'), ('7', 'B-NO'), ('permanent', 'O'), ('address', 'O'), ('district', 'O'), ('makawanpur', 'I-DISTRICT'), ('municipality', 'O'), ('thaha', 'I-WARD'), ('ward', 'O'), ('no', 'I-NO'), ('7', 'I-NO'), ('hare', 'O'), ('aaa', 'O'), ('taf', 'O'), ('083', 'I-NO'), ('atta', 'O'), ('at', 'O'), ('a', 'O'), ('reca', 'O'), ('tat', 'O'), ('rec', 'O'), ('feafear', 'O'), ('aerst', 'O'), ('tf', 'O')]


In [14]:
# if tags are more then one take only first,  except for B-No and I -NO . for them consider two.


newnew_words = []
tagss=[]
for idx , (word, label) in enumerate(new_words):
    tagss.append(label)
    if label not in ['O']:
        if label in ["B-NO", "I-NO"]:
                if tagss.count(label) > 2:
                    continue
        elif tagss.count(label) > 1:
            continue
    newnew_words.append((word, label))


print(newnew_words)


[('details', 'O'), ('citizenship', 'O'), ('certificate', 'O'), ('no', 'O'), ('37027700054', 'CIT-NUM'), ('sex', 'O'), ('male', 'SEX'), ('full', 'O'), ('names', 'O'), ('sagargiri', 'B-NAME'), ('i', 'O'), ('date', 'O'), ('of', 'O'), ('birth', 'O'), ('ad', 'O'), ('year', 'O'), ('2003', 'YEAR'), ('month', 'O'), ('jun', 'MONTH'), ('day', 'O'), ('06', 'DAY'), ('birth', 'O'), ('placs', 'O'), ('cds', 'O'), ('trict', 'O'), ('va', 'O'), ('srniranra', 'I-NAME'), ('dees', 'O'), ('tet', 'O'), ('rh', 'O'), ('vdc', 'O'), ('bamnghs', 'B-WARD'), ('ward', 'O'), ('no', 'B-NO'), ('8', 'B-NO'), ('i', 'O'), ('perman', 'O'), ('nt', 'O'), ('address', 'O'), ('s', 'O'), ('district', 'O'), ('rupandehi', 'B-DISTRICT'), ('sly', 'O'), ('pisare', 'I-DISTRICT'), ('ce', 'O'), ('igs', 'O'), ('yl', 'O'), ('municipality', 'O'), ('tilottama', 'I-WARD'), ('ward', 'O'), ('no', 'I-NO'), ('1', 'I-NO'), ('boe', 'O'), ('aia', 'O'), ('ca', 'O'), ('aig', 'O'), ('at', 'O'), ('aera', 'O'), ('watt', 'O')]


In [15]:


result = {}
full_name = []
citizenship_no = ""
gender = ""
year = ""
month = ""
day = ""
B_dist = ""
B_ward = ""
B_no = ""
P_dist = ""
P_ward = ""
P_no = ""

def extract_num(text):
    match = re.search(r'\d+', text)
    return match.group() if match else None


for word, label in newnew_words:
    if label == "CIT-NUM":
        citizenship_no += word
    elif label == "B-NAME":
        full_name.append(word)
    elif label == "I-NAME":
        full_name.append(word)
    elif label == "SEX":
      gender +=word
    elif label == "YEAR":
      year += word
    elif label == "MONTH":
      month += word
    elif label == "DAY":
      day += word
    elif label == "B-DISTRICT":
      B_dist += word
    elif label == "B-WARD":
      B_ward += word
    elif label == "B-NO":
      B_no += word
    elif label == "I-DISTRICT":
      P_dist += word
    elif label == "I-WARD":
      P_ward += word
    elif label == "I-NO":
      P_no += word

if citizenship_no:
    citizenship_number = citizenship_no[:2] + "-" + citizenship_no[2:4] + "-" + citizenship_no[4:6] + "-" + citizenship_no[6:]
    result["citizenship_num"] = citizenship_number
else :
  result["citizenship_num"] = None

if len(full_name) > 0:
    result["first_name"] = full_name[0]
else:
   result["first_name"] = None

if len(full_name) > 1:
    result["last_name"] = full_name[-1]
else:
  result["last_name"] = None
if len(gender)>1:
    result["gender"] = gender
else:
  result["gender"] = None
if year:
    result["Birth_year"] = year
    result["Birth_year"] = extract_num(result["Birth_year"])
else:
  result["Birth_year"] = None

if len(month) >1:
    result["Birth_month"] = month
else:
  result["Birth_month"] = None
if day:
    result["Birth_day"] = day
else:
  result["Birth_day"] = None
if B_dist:
    result["Birth_district"] = B_dist
else:
  result["Birth_district"] = None
if B_ward:
    result["Birth_ward"] = B_ward
else:
  result["Birth_ward"] = None
if B_no:
    result["Birth_wardno"] = B_no
    result["Birth_wardno"] = extract_num(result["Birth_wardno"])
else:
  result["Birth_wardno"] = None
if P_dist:
    result["Permanent_dist"] = P_dist
else:
  result["Permanent_dist"] = None
if P_ward:
    result["Permanent_ward"] = P_ward
else:
  result["Permanent_ward"] = None
if P_no:
    result["Permanent_wardno"] = P_no
    result["Permanent_wardno"] = extract_num(result["Permanent_wardno"])
else:
  result["Permanent_wardno"] = None


print(result)

{'citizenship_num': '37-02-77-00054', 'first_name': 'sagargiri', 'last_name': 'srniranra', 'gender': 'male', 'Birth_year': '2003', 'Birth_month': 'jun', 'Birth_day': '06', 'Birth_district': 'rupandehi', 'Birth_ward': 'bamnghs', 'Birth_wardno': '8', 'Permanent_dist': 'pisare', 'Permanent_ward': 'tilottama', 'Permanent_wardno': '1'}
